<a href="https://colab.research.google.com/github/SiMori92/reddit-NLP/blob/main/Keywords_time_series_binary_v3_(post_submit_revision).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Problem we solve:
- Correlation between Reddit comment and Stock market performance

Content

- Pre1: load library
- Pre2: load data
  - split training set & testing set
1. Training
  1. models
    - SVM
    - Gaussian Naive Bayes
    - Random Forest
    - XGBoost
    - KNN
    - Perceptron
    - Stochastic Gradient Descent (SGD)
2. Evaluation
  1. Accuracy
  2. F1
3. Analysis

In [ ]:
# import library

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import re
import string

%matplotlib inline

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Libraries for text preprocessing.
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.metrics import precision_score, recall_score, f1_score

!pip install bert-tensorflow
from bert import tokenization

from wordcloud import STOPWORDS

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 1.2 MB/s eta 0:00:00


In [ ]:
# #load excel

consol_df = pd.read_csv("/content/drive/MyDrive/CUSCS - Group project/Reddit_WhatAreYourMoveTmr/consol_Reddit_comment_Jan-Jun23.csv")
consol_df = consol_df.set_index('date')
consol_df = consol_df.drop(columns=['Unnamed: 0', 'index'])
consol_df


,comments,Open_x,High_x,Low_x,Close_x,Adj Close_x,Volume_x,percent_chnage_x,Jump_x,Big_Jump_x,...,High_y,Low_y,Close_y,Adj Close_y,Volume_y,percent_chnage_y,Jump_y,Big_Jump_y,Drop_y,Big_Drop_y
date,,,,,,,,,,,,,,,,,,,,,
2022-07-13,This week's [Earnings Thread](https://www.redd...,3779.669922,3829.439941,3759.070068,3801.780029,3801.780029,4109390000,0.584975,1,0,...,11325.66992,11031.26953,11247.58008,11247.58008,4433060000,1.727757,0,1,0,0
2022-07-14,This week's [Earnings Thread](https://www.redd...,3763.989990,3796.409912,3721.560059,3790.379883,3790.379883,4199690000,0.701115,1,0,...,11279.96973,11005.92969,11251.19043,11251.19043,4481070000,0.896589,1,0,0,0
2022-07-15,Cashed out up 56k today. Now cuddled up watchi...,3818.000000,3863.620117,3817.179932,3863.159912,3863.159912,4143800000,1.182816,0,1,...,11454.69043,11295.33008,11452.41992,11452.41992,4369060000,0.642036,1,0,0,0
2022-07-19,This week's [Earnings Thread](https://www.redd...,3860.729980,3939.810059,3860.729980,3936.689941,3936.689941,4041070000,1.967503,0,1,...,11721.21973,11448.96973,11713.15039,11713.15039,5302740000,1.720802,0,1,0,0
2022-07-20,If 2008 was the Great Recession\n\nThen 2022 i...,3935.320068,3974.129883,3922.030029,3959.899902,3959.899902,4185300000,0.624596,1,0,...,11939.95996,11703.36035,11897.65039,11897.65039,5467080000,1.463067,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-26,I was holding puts over the weekend. Thought f...,4344.839844,4362.060059,4328.080078,4328.819824,4328.819824,3415030000,-0.368714,0,0,...,13573.57031,13334.41992,13335.78027,13335.78027,4430600000,-0.987174,0,0,1,0
2023-06-27,I watched TSLA rise for 3 months. 3 months. 3 ...,4337.359863,4384.419922,4335.000000,4378.410156,4378.410156,3573500000,0.946435,1,0,...,13578.79980,13366.96973,13555.66992,13555.66992,5053660000,1.242937,0,1,0,0
2023-06-28,Trades when I 15x: $300 invested\n\nTrades whe...,4367.479980,4390.350098,4360.220215,4376.859863,4376.859863,3739330000,0.214766,1,0,...,13654.13965,13495.73047,13591.75000,13591.75000,4533270000,0.634757,1,0,0,0


In [ ]:
column_names = consol_df.columns.tolist()

print(column_names)

consol_df = consol_df.rename(columns={'Volume_x': 'Volume_SP500','Jump_x':'Jump_SP500','Big_Jump_x':'Big_Jump_SP500','Drop_x':'Drop_SP500','Big_Drop_x':'Big_Drop_SP500',
                      'Volume_y': 'Volume_NQ','Jump_y':'Jump_NQ','Big_Jump_y':'Big_Jump_NQ','Drop_y':'Drop_NQ','Big_Drop_y':'Big_Drop_NQ'})

['comments', 'Open_x', 'High_x', 'Low_x', 'Close_x', 'Adj Close_x', 'Volume_x', 'percent_chnage_x', 'Jump_x', 'Big_Jump_x', 'Drop_x', 'Big_Drop_x', 'Open_y', 'High_y', 'Low_y', 'Close_y', 'Adj Close_y', 'Volume_y', 'percent_chnage_y', 'Jump_y', 'Big_Jump_y', 'Drop_y', 'Big_Drop_y']


## 4. Training

In [ ]:
# training models

# LSTM
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

# Gaussian Naive Bayes
from sklearn.naive_bayes import GaussianNB

# SVM
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.multioutput import MultiOutputClassifier

# Random Forest
from sklearn.ensemble import RandomForestClassifier

# XGBoost
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV

# KNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier

# Perceptron
from sklearn.linear_model import Perceptron

# Stochastic Gradient Descent (SGD)
from sklearn.linear_model import SGDClassifier

In [ ]:
# Gaussian Naive Bayes

nb_clf = GaussianNB()
nb_clf.fit(X_train.values, NQ_train.values)

GaussianNB()

In [ ]:
# SVM
# SVM doesn't support multiclass classification natively

clf_svm = SVC(kernel='linear', C=1, max_iter=500)
clf_svm.fit(X_train.values, NQ_train.values)

SVC(C=1, kernel='linear', max_iter=500)

In [ ]:
# Random Forest

# the parameters meaning
#  n_estimators 随机森林中树的个数，即学习器的个数。
# n_jobs 并行使用的进程数，默认1个，如果设置为-1，该值为总的核数
# max_depth 树的最大深度
# random_state =42 just a common practice

classifier_rf = RandomForestClassifier(random_state=42, n_jobs=-1, max_depth=15,
                                       n_estimators=500)

# Random Forest trained with NQ

classifier_rf.fit(X_train.values, NQ_train.values)

RandomForestClassifier(max_depth=15, n_estimators=500, n_jobs=-1,
                       random_state=42)

In [ ]:
# Random Forest with XGB

gbm_param_grid = {
    'n_estimators': range(8, 300),
    'max_depth': range(6, 20),
    'learning_rate': [.4, .45, .5, .55, .6],
    'colsample_bytree': [.6, .7, .8, .9, 1]
}
gbm = XGBClassifier(n_estimators=300)
xgb_random = RandomizedSearchCV(param_distributions=gbm_param_grid,
                                    estimator = gbm, scoring = "accuracy",
                                    verbose = 1, n_iter = 50, cv = 4)
# Random Forest with XGB trained with NQ

xgb_random.fit(X_train.values, NQ_train.values)

Fitting 4 folds for each of 50 candidates, totalling 200 fits


RandomizedSearchCV(cv=4,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, gpu_id=None,
                                           grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate...
                                           max_leaves=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n_estimators=300, n_jobs=None,
                                           num_parallel_tree=None,
                                           predictor=None, random_state=None, ...),
                   n_iter=50,
                   param_distributions={'colsample_bytree': [0.6, 0.7, 0.8, 0.9,
                                                             1],
                                        'learning_rate': [0.4, 0.45, 0.5, 0.55,
                                                          0.6],
                                        'max_depth': range(6, 20),
                                        'n_estimators': range(8, 300)},
                   scoring='accuracy', verbose=1)

In [ ]:
# KNeighborsClassifier

knn_clf = KNeighborsClassifier(n_neighbors = 10)
knn_clf.fit(X_train.values, NQ_train.values)

KNeighborsClassifier(n_neighbors=10)

In [ ]:
# Perceptron

per_clf = Perceptron(max_iter=10, tol=None)
per_clf.fit(X_train.values, NQ_train.values)

Perceptron(max_iter=10, tol=None)

In [ ]:
# Stochastic Gradient Descent (SGD)

sgd_clf = SGDClassifier(max_iter=10, tol=None)
sgd_clf.fit(X_train.values, NQ_train.values)

SGDClassifier(max_iter=10, tol=None)

## 5. Evaluation

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
# Gaussian Naive Bayes
y_predict_bayes = nb_clf.predict(X_test.values)

print(classification_report(NQ_test,y_predict_bayes))

              precision    recall  f1-score   support

           0       0.25      0.80      0.38         5
           1       0.50      0.08      0.13        13

    accuracy                           0.28        18
   macro avg       0.38      0.44      0.26        18
weighted avg       0.43      0.28      0.20        18



In [ ]:
# SVM

y_predict_svm = clf_svm.predict(X_test.values)

print(classification_report(NQ_test,y_predict_svm))

              precision    recall  f1-score   support

           0       0.25      0.20      0.22         5
           1       0.71      0.77      0.74        13

    accuracy                           0.61        18
   macro avg       0.48      0.48      0.48        18
weighted avg       0.59      0.61      0.60        18



In [ ]:
# Random forest
y_predict_rf = classifier_rf.predict(X_test.values)

print(classification_report(NQ_test,y_predict_rf))

              precision    recall  f1-score   support

           0       0.50      0.20      0.29         5
           1       0.75      0.92      0.83        13

    accuracy                           0.72        18
   macro avg       0.62      0.56      0.56        18
weighted avg       0.68      0.72      0.68        18



In [ ]:
# Random Forest with XGB

y_predict_xgb = xgb_random.predict(X_test.values)

print(classification_report(NQ_test,y_predict_xgb))

              precision    recall  f1-score   support

           0       0.50      0.40      0.44         5
           1       0.79      0.85      0.81        13

    accuracy                           0.72        18
   macro avg       0.64      0.62      0.63        18
weighted avg       0.71      0.72      0.71        18



In [ ]:
# KNN
y_predict_KNN = knn_clf.predict(X_test.values)

print(classification_report(NQ_test,y_predict_KNN))

              precision    recall  f1-score   support

           0       0.40      0.80      0.53         5
           1       0.88      0.54      0.67        13

    accuracy                           0.61        18
   macro avg       0.64      0.67      0.60        18
weighted avg       0.74      0.61      0.63        18



In [ ]:
# Perceptron

y_predict_per = per_clf.predict(X_test.values)
print(classification_report(NQ_test,y_predict_per))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.71      0.92      0.80        13

    accuracy                           0.67        18
   macro avg       0.35      0.46      0.40        18
weighted avg       0.51      0.67      0.58        18



In [ ]:
#Stochastic Gradient Descent (SGD)

y_predict_sgd = sgd_clf.predict(X_test.values)

print(classification_report(NQ_test,y_predict_sgd))

              precision    recall  f1-score   support

           0       0.24      0.80      0.36         5
           1       0.00      0.00      0.00        13

    accuracy                           0.22        18
   macro avg       0.12      0.40      0.18        18
weighted avg       0.07      0.22      0.10        18



## 6. Analysis

- Accuracy: 0.72 Random Forest vs 0.28 Bayes
-------------
- Precision of down (0) :0.55 Random Forest vs 0 Perceptron & SGD
- Precision of up (1) : 0.88 KNN vs 0.55 Random Forest
-------------
- F1-score ( weighted average of precision and recall) of down (0): 0.53 KNN vs 0 Perceptron & SGD
- F1-score ( weighted average of precision and recall) of up (1): 0.83 Random Forest vs 0.13 Bayes

Overall: Random forest vs KNN



In [ ]:
# if doing AND to Random forest vs KNN

y_predict_combined_and = np.logical_and(y_predict_rf, y_predict_KNN).astype(int)
y_predict_combined_and

print(classification_report(NQ_test,y_predict_combined_and))

              precision    recall  f1-score   support

           0       0.40      0.80      0.53         5
           1       0.88      0.54      0.67        13

    accuracy                           0.61        18
   macro avg       0.64      0.67      0.60        18
weighted avg       0.74      0.61      0.63        18



In [ ]:
# if doing OR to Random forest vs KNN

y_predict_combined_or = np.logical_or(y_predict_rf, y_predict_KNN).astype(int)
y_predict_combined_or

print(classification_report(NQ_test,y_predict_combined_or))

              precision    recall  f1-score   support

           0       0.50      0.20      0.29         5
           1       0.75      0.92      0.83        13

    accuracy                           0.72        18
   macro avg       0.62      0.56      0.56        18
weighted avg       0.68      0.72      0.68        18

